# Getting timestamps on long audio

> "With wav2vec2 and HuggingFace transformers"

- tok: false
- branch: master
- comments: true
- categories: [long audio, wav2vec2, huggingface, timestamps]


First, an audio sample. Using [this video](https://www.youtube.com/watch?v=Kw5jkyLGFGc) from youtube. Youtube says it's 11 minutes, 51 seconds, so that should be enough to check that striding works.

In [ ]:
!pip install youtube-dl

In [ ]:
!youtube-dl -x --audio-format best -o '%(id)s.%(ext)s' https://www.youtube.com/watch?v=Kw5jkyLGFGc

In [ ]:
!ffmpeg -i Kw5jkyLGFGc.m4a -acodec pcm_s16le -ac 1 -ar 16000 Kw5jkyLGFGc.wav

Here starts the actual ASR stuff.

In [ ]:
!pip install transformers

In [5]:
_SWE_MODEL = "KBLab/wav2vec2-large-voxrex-swedish"

In [6]:
from transformers import pipeline

In [ ]:
pipe = pipeline(model=_SWE_MODEL)

For working with strides, there's information in a [blog post](https://huggingface.co/blog/asr-chunking).

There isn't much information on getting timestamps from a pipeline, but the detail is in the [pull request](https://github.com/huggingface/transformers/pull/15792).

In [8]:
output = pipe("/content/Kw5jkyLGFGc.wav", chunk_length_s=10)

In [9]:
output = pipe("/content/Kw5jkyLGFGc.wav", chunk_length_s=10, return_timestamps="word")

In [10]:
import json
with open("/content/Kw5jkyLGFGc.json", "w") as f:
    json.dump(output, f)